In [ ]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.LightGbm"


In [ ]:
public class DummyData
{
    public float Feature1 { get; set; }
    public float Feature2 { get; set; }
    public float Feature3 { get; set; }
    public float Feature4 { get; set; }
    public bool Label { get; set; }
}

In [ ]:
var mlContext = new Microsoft.ML.MLContext();
var dataPoints = GenerateRandomDataPoints(1000).ToList();
var allData = mlContext.Data.LoadFromEnumerable(dataPoints);

// Split the data into train and test set
var trainTestSplit = mlContext.Data.TrainTestSplit(allData);
var trainingData = trainTestSplit.TrainSet;
var testData = trainTestSplit.TestSet;

IEnumerable<DummyData> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f;
        yield return new DummyData
        {
            Label = label,
            Feature1 = randomFloat(),
            Feature2 = randomFloat(),
            Feature3 = randomFloat(),
            Feature4 = randomFloat()
        };
    }
}

Console.WriteLine($"Number of training examples: {trainingData.GetRowCount()}");
Console.WriteLine($"Number of test examples: {testData.GetRowCount()}");

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;

var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Feature1", "Feature2", "Feature3", "Feature4" })
                .Append(mlContext.BinaryClassification.Trainers.LightGbm());

Console.WriteLine("Pipeline created successfully");

In [ ]:
// Train the model using the pipeline and training data
var model = pipeline.Fit(trainingData);

// Make predictions with the trained model and test data
var predictions = model.Transform(testData);

// Evaluate the model's quality
var metrics = mlContext.BinaryClassification.Evaluate(predictions);

// Print out the result to console
Console.WriteLine($"Accuracy: {metrics.Accuracy}");
Console.WriteLine($"Auc: {metrics.AreaUnderRocCurve}");
Console.WriteLine($"F1Score: {metrics.F1Score}");

In [ ]:
mlContext.Model.Save(model, trainingData.Schema, "lgbm.mlnet");

Console.WriteLine("Model saved to lgbm.mlnet");